In [ ]:
import json
import pandas as pd
import numpy as np
from difflib import SequenceMatcher

In [ ]:
NLI_FILE = "../result/private_test_retrieval_v1_top5_top_5_mDeBERTa (ft) V6(mean).json"
CHECKER_FILE = "../result/private_test_retrieval_v1_top5_top_5_checked.json"

In [ ]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [ ]:
nli_result = json.load(open(NLI_FILE))
checker_result = json.load(open(CHECKER_FILE))

In [ ]:
for key in nli_result.keys():
    nli_result[key]['similarity'] = list()
    for evidence in nli_result[key]['evidence']:
        nli_result[key]['similarity'].append(similar(nli_result[key]['claim'], evidence))

In [ ]:
for key in nli_result.keys():
    if set(nli_result[key]['verdict']) == {'NEI'}:
        nli_result[key]['verdict'] = 'NEI'
        nli_result[key]['evidence'] = ''
    else:
        result_list = [
            (nli_result[key]['verdict'][x],
                nli_result[key]['evidence'][x],
                nli_result[key]['prob'][x],
                nli_result[key]['similarity'][x],
                ) for x in range(len(nli_result[key]['verdict'])) if nli_result[key]['verdict'][x] != "NEI"]
        
        max_prob_index  = np.argmax([x[2] for x in result_list])
        max_similar_index  = np.argmax([x[3] for x in result_list])
                
        nli_result[key]['verdict'] = result_list[max_similar_index][0]
        nli_result[key]['evidence'] = result_list[max_similar_index][1]

In [ ]:
for key in checker_result.keys():
    if 'SUPPORTED' in checker_result[key]['verdict']:
        evidence_index = checker_result[key]['verdict'].index('SUPPORTED')
        checker_result[key]['verdict'] = 'SUPPORTED'
        checker_result[key]['evidence'] = checker_result[key]['evidence'][evidence_index]
    
    elif 'REFUTED' in checker_result[key]['verdict']:
        evidence_index = checker_result[key]['verdict'].index('REFUTED')
        checker_result[key]['verdict'] = 'REFUTED'
        checker_result[key]['evidence'] = checker_result[key]['evidence'][evidence_index]
    
    else:
        checker_result[key]['verdict'] = ''
        checker_result[key]['evidence'] = ''
        continue

In [ ]:
private_test_result = dict()

In [ ]:
for key in nli_result.keys():
    private_test_result[key] = dict()
    private_test_result[key]['context'] = nli_result[key]['context']
    private_test_result[key]['claim'] = nli_result[key]['claim']
    if checker_result[key]['verdict'] != '':
        private_test_result[key]['verdict'] = checker_result[key]['verdict']
        private_test_result[key]['evidence'] = checker_result[key]['evidence']
    else:
        private_test_result[key]['verdict'] = nli_result[key]['verdict']
        private_test_result[key]['evidence'] = nli_result[key]['evidence']

In [ ]:
with open(f"../result/private_submit_v3_mean_v1/private_result.json", "w") as outfile:
    json.dump(private_test_result, outfile)